In [1]:
import datetime

import matplotlib.pyplot as plt

from acsql.database.database_interface import session
from acsql.database.database_interface import WFC_raw_0
from acsql.database.database_interface import HRC_raw_0
from acsql.database.database_interface import SBC_raw_0

In [2]:
def get_dates(detector):
    """Return a list of observation dates and the aggregate number of
    observations for every observation for the given detector over time.
    
    Parameters
    ----------
    detector : obj
        The SQLAlchemy Table object for the detector.
        
    Returns
    -------
    dates : list
        A list of observation dates
    num_obs : list
        A list of the aggregate number of observations over time for each
        date in the dates list.
    """
    
    results = session.query(getattr(detector,'date-obs')).all()
    dates = sorted([item[0] for item in results])
    results_dict = {}
    num_obs = 1
    for date in dates:
        results_dict[date] = num_obs
        num_obs += 1
    dates, num_obs = zip(*sorted(results_dict.items()))
    dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

    return dates, num_obs

In [3]:
wfc_dates, wfc_num_obs = get_dates(WFC_raw_0)
hrc_dates, hrc_num_obs  = get_dates(HRC_raw_0)
sbc_dates, sbc_num_obs  = get_dates(SBC_raw_0)

In [4]:
plt.rcParams['font.size'] = 14
plt.style.use('bmh')
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.set_ylabel('Number of Observations')
ax.set_yticks([0, 20000, 40000, 60000, 80000, 100000, 120000])
ax.set_yticklabels(['0', '20k', '40k', '60k', '80k', '100k', '120k'])
ax.plot(wfc_dates, wfc_num_obs, linewidth=3, color='#348ABD', label='WFC')
ax.plot(hrc_dates, hrc_num_obs, linewidth=3, color='#A60628', label='HRC')
ax.plot(sbc_dates, sbc_num_obs, linewidth=3, color='#467821', label='SBC')
plt.legend()
plt.tight_layout()
plt.savefig('figures/num_obs.png')